In [1]:
!pip install -U datasets sentence-transformers pinecone-client tqdm

In [2]:
from datasets import load_dataset

ytt = load_dataset(
    "pinecone/yt-transcriptions",
    split="train",
    revision="926a45"
)
ytt

Dataset({
    features: ['video_id', 'text', 'start_second', 'end_second', 'url', 'title', 'thumbnail'],
    num_rows: 11298
})

In [3]:
for x in ytt:
    print(x)
    break

{'video_id': 'ZPewmEu7644', 'text': " hi this is Jeff Dean welcome to applications of deep neural networks of Washington University in this video we're going to look at how we can use ganz to generate additional training data for the latest on my a I course and projects click subscribe in the bell next to it to be notified of every new video Dan's have a wide array of uses beyond just the face generation that you", 'start_second': 0, 'end_second': 20, 'url': 'https://www.youtube.com/watch?v=ZPewmEu7644&t=0s', 'title': 'GANS for Semi-Supervised Learning in Keras (7.4)', 'thumbnail': 'https://i.ytimg.com/vi/ZPewmEu7644/maxresdefault.jpg'}


In [4]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

retriever = SentenceTransformer(
    'flax-sentence-embeddings/all_datasets_v3_mpnet-base'
)
retriever.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [5]:
embed_dim = retriever.get_sentence_embedding_dimension()
embed_dim

768

In [9]:
from pinecone import Pinecone

# get api key from app.pinecone.io
pinecone = Pinecone(
    api_key="55e1c7a7-6374-4a46-9650-b075f74e2158",
    environment="us-west1-gcp"
)

index = pinecone.Index("youtube-search")

In [24]:
query = "what is tranformers"

xq = retriever.encode([query]).tolist()

In [25]:
xc = index.query(vector=xq, top_k=20,
                 include_metadata=True)
for context in xc['matches']:
    print(context['metadata']['url'], end="\n---\n")

https://www.youtube.com/watch?v=-9evrZnBorM&t=101s
---
https://www.youtube.com/watch?v=Uumd2zOOz60&t=1164s
---
https://www.youtube.com/watch?v=H5vpBCLo74U&t=1255s
---
https://www.youtube.com/watch?v=T35ba_VXkMY&t=1246s
---
https://www.youtube.com/watch?v=H5vpBCLo74U&t=1438s
---
https://www.youtube.com/watch?v=T35ba_VXkMY&t=1160s
---
https://www.youtube.com/watch?v=Uumd2zOOz60&t=114s
---
https://www.youtube.com/watch?v=H5vpBCLo74U&t=1470s
---
https://www.youtube.com/watch?v=LfUsGv-ESbc&t=326s
---
https://www.youtube.com/watch?v=hv3UO3G0Ofo&t=1025s
---
https://www.youtube.com/watch?v=hv3UO3G0Ofo&t=989s
---
https://www.youtube.com/watch?v=Uumd2zOOz60&t=591s
---
https://www.youtube.com/watch?v=T35ba_VXkMY&t=0s
---
https://www.youtube.com/watch?v=T35ba_VXkMY&t=1802s
---
https://www.youtube.com/watch?v=T35ba_VXkMY&t=1217s
---
https://www.youtube.com/watch?v=BnpB3GrpsfM&t=5124s
---
https://www.youtube.com/watch?v=T35ba_VXkMY&t=1851s
---
https://www.youtube.com/watch?v=T35ba_VXkMY&t=1377s
---


In [20]:
print(xc)

{'matches': [{'id': 'BnpB3GrpsfM-6551',
              'metadata': {'end_second': 6573.0,
                           'start_second': 6551.0,
                           'text': " performance on a data set so that's quite "
                                   'exciting and in the limit they show that '
                                   'for instance in Elektra a small electrical '
                                   'model so quite a lot smaller than even a '
                                   'GPT one by exploiting bidirectionality in '
                                   'this dense training function can actually '
                                   'outperform GPT one in two days on a single '
                                   'be 100 whereas gbg1 was 25 days on 8p 6000 '
                                   'partially this is because I have P',
                           'thumbnail': 'https://i.ytimg.com/vi/BnpB3GrpsfM/maxresdefault.jpg',
                           'title': 'L11 Language